In [1]:
!pip install pydub

  Obtaining dependency information for pydub from https://files.pythonhosted.org/packages/a6/53/d78dc063216e62fc55f6b2eebb447f6a4b0a59f55c8406376f76bf959b08/pydub-0.25.1-py2.py3-none-any.whl.metadata


In [1]:
import openai
import os
import re
import json
from pydub import AudioSegment

In [2]:
openai.api_key = os.getenv('OPEN_AI_KEY') # captura a API-key salva no ambiente de sistema

## Importa o audio da live

In [3]:
path_audio = "Y:\Transcription lives\CONHEÇA O ULTIMO JOGO DE FORMULA INDY OFICIALMENTE LANÇADO - IndyCar Series 2005 (PS2).mp3"

In [4]:
audio_file = open(path_audio, "rb")

## divida o arquivo em 10 partes iguais

In [5]:
partes = []

# Carregar o arquivo de áudio
audio = AudioSegment.from_file(path_audio)

# Duração total do áudio em milissegundos
duration = len(audio)

# Duração de cada parte do áudio
part_duration = duration // 10

# Dividir o áudio em 10 partes e salvar
for part in range(10):
    start_time = part * part_duration
    end_time = (part + 1) * part_duration if (part < 9) else duration
    new_clip = audio[start_time:end_time]
    partes.append(new_clip)
    new_clip.export(f"Y:\Transcription lives\data\part_{part+1}.mp3", format="mp3")
    print(f"Parte {part+1} do audio exportada com sucesso")
    
# # Função para extrair o número da parte do nome do arquivo
# def extract_part_number(file_path):
#     match = re.search(r'part_(\d+)', file_path)
#     return int(match.group(1)) if match else 0

# # Ordenar a lista usando a função personalizada
# sorted_file_paths = sorted(partes, key=extract_part_number)
# file_paths = partes.copy()

Parte 1 do audio exportada com sucesso
Parte 2 do audio exportada com sucesso
Parte 3 do audio exportada com sucesso
Parte 4 do audio exportada com sucesso
Parte 5 do audio exportada com sucesso
Parte 6 do audio exportada com sucesso
Parte 7 do audio exportada com sucesso
Parte 8 do audio exportada com sucesso
Parte 9 do audio exportada com sucesso
Parte 10 do audio exportada com sucesso


In [6]:
partes

In [7]:
# Diretório onde os arquivos MP3 estão localizados
directory = 'Y:/Transcription lives/data'

# Lista para armazenar os caminhos dos arquivos MP3
mp3_files = []

# Listar todos os arquivos no diretório
for file in os.listdir(directory):
    if file.endswith(".mp3"):
        mp3_files.append(os.path.join(directory, file))

## Exportando a transcrição de cada parte
Uma live de 2h vai custar aproximadamente $0,60 USD ~ $0,80 pra transcrever.

In [8]:
transcricoes = {}

for mp3_file in mp3_files:
    audio_file = open(mp3_file, "rb")
    file_name = mp3_file.split('\\')[-1]
    transcript = openai.Audio.transcribe("whisper-1", audio_file, response_format="verbose_json")
    audio_transcription = transcript['text']
    transcricoes[file_name] = audio_transcription
    print(f"Audio {file_name} transcrito com sucesso")

Audio part_1.mp3 transcrito com sucesso
Audio part_10.mp3 transcrito com sucesso
Audio part_2.mp3 transcrito com sucesso
Audio part_3.mp3 transcrito com sucesso
Audio part_4.mp3 transcrito com sucesso
Audio part_5.mp3 transcrito com sucesso
Audio part_6.mp3 transcrito com sucesso
Audio part_7.mp3 transcrito com sucesso
Audio part_8.mp3 transcrito com sucesso
Audio part_9.mp3 transcrito com sucesso


In [9]:
transcricoes

{'part_1.mp3': '♪ ♪ ♪ ♪ Pra você ligado aqui no Fla Gamer, muito boa noite! E pra você, hoje nós temos Live Retro, é, Live Retro de novo, com IndyCar Series 2005. Esse simplesmente foi o último jogo oficial lançado da Fórmula Indy. Já vai fazer 20 anos ano que vem, ele foi lançado em 2004, 2004, por a volta disso, representando a temporada da IRL de 2003. Um jogo desenvolvido aí pela Codemasters, essa segunda versão que teve o IndyCar Series, o primeiro que até saiu pro PC, e esse segundo só saiu pro Playstation 2 e pro Xbox. E... ele é um jogo feito pela Codemasters e é um jogo difícil. Não vou confessar pra vocês não, hoje vocês vão ver aqui que o Fla Gamer, se ele completar uma corrida, vai ser um milagre. Porque ele é realmente um jogo que tende pro simulador, tanto é que eu tô de volante, consegui configurar o volante pro emulador do Playstation 2. Aliás, fica a dica, trava o volante em 180 graus quando tu for configurar pro emulador do Playstation 2, tá? Que era mais ou menos iss

## Exportar o dicionário para um arquivo JSON

In [11]:
with open("Y:/Transcription lives/data/transcription.json", 'w', encoding='utf-8') as json_file:
    json.dump(transcricoes, json_file, ensure_ascii=False, indent=4)

## Agora vai resumir cada parte e exportar em texto
Resumir as 10 partes da live custou aproximadamente USD 0,30

In [13]:
system_msg = "Sua missão é resumir e listar os principais pontos que o Flagamer mencionou em cada parte da live em forma de um texto impessoal e predominantemente descritivo. Se não conseguir transcrever, apenas retorne Infelizmente esta parte não pode ser transcrita"

In [14]:
output_texto = open("Y:\Transcription lives\data\output.txt", "w")

for chave, valor in transcricoes.items():
    user_msg = f"{valor}"
    output_texto.write(f"Resumo {chave}\n:")
    response = openai.ChatCompletion.create(model="gpt-4",
                                       messages=[{"role": "system", "content":system_msg},
                                                {"role": "user", "content":user_msg}],
                                       temperature=0.25)
    output_texto.write(response["choices"][0]["message"]["content"])
    output_texto.write("\n")
    print(f"Resumo da parte {chave} gerado com sucesso")
    
output_texto.close()
print("Resumo da live gerado com sucesso")

Resumo da parte part_1.mp3 gerado com sucesso
Resumo da parte part_10.mp3 gerado com sucesso
Resumo da parte part_2.mp3 gerado com sucesso
Resumo da parte part_3.mp3 gerado com sucesso
Resumo da parte part_4.mp3 gerado com sucesso
Resumo da parte part_5.mp3 gerado com sucesso
Resumo da parte part_6.mp3 gerado com sucesso
Resumo da parte part_7.mp3 gerado com sucesso
Resumo da parte part_8.mp3 gerado com sucesso
Resumo da parte part_9.mp3 gerado com sucesso
Resumo da live gerado com sucesso


## Agora vai pegar essas 10 partes e fazer um texto corrido

In [17]:
output_texto = open("Y:\Transcription lives\data\output.txt", "r")
texto_corrido = output_texto.readlines()
texto = ''.join(texto_corrido)

In [18]:
texto

'Resumo part_1.mp3\n:Flagamer inicia a live retro com o jogo "IndyCar Series 2005", destacando que este foi o último jogo oficial da Fórmula Indy, lançado em 2004 e representando a temporada de 2003 da IRL. Desenvolvido pela Codemasters, o jogo é descrito como difícil e tendendo ao simulador, com gerenciamento de combustível e ajustes no equilíbrio do carro.\n\nEle menciona que está jogando com um volante configurado para o emulador do Playstation 2, recomendando travar o volante em 180 graus para uma experiência mais autêntica. Agradece aos membros do canal e convida novos espectadores a se tornarem membros, destacando benefícios como participação em corridas exclusivas com premiação de gift cards mensais.\n\nFlagamer responde a uma pergunta sobre um novo jogo da Fórmula Indy que não será lançado devido a problemas financeiros da empresa desenvolvedora. Ele explica as configurações iniciais do jogo, como ajuste de sensibilidade do volante e configuração da tela para 16:9.\n\nO jogo po

In [27]:
system_msg = "Sua missão é fazer um texto corrido de até 2000 caracteres sobre os principais pontos das 10 partes da live com estilo de texto descritivo. A parte 10 da live deve ser obrigatoriamente o parágrafo final do texto"

In [28]:
user_msg = f"{texto}"
response = openai.ChatCompletion.create(model="gpt-4o",
                                   messages=[{"role": "system", "content":system_msg},
                                            {"role": "user", "content":user_msg}],
                                   temperature=0.6)

In [29]:
print(response["choices"][0]["message"]["content"])

A live de Flagamer dedicada ao jogo "IndyCar Series 2005" começou com uma introdução ao contexto histórico do jogo, que foi o último título oficial da Fórmula Indy, lançado em 2004 e representando a temporada de 2003 da IRL. Desenvolvido pela Codemasters, o jogo é conhecido por sua dificuldade e tendência ao simulador, exigindo gerenciamento de combustível e ajustes no equilíbrio do carro. Flagamer destacou a configuração do volante para o emulador do Playstation 2, recomendando travá-lo em 180 graus para uma experiência mais autêntica. Ele também agradeceu aos membros do canal e incentivou novos espectadores a se tornarem membros, destacando benefícios como participação em corridas exclusivas com premiação de gift cards mensais.

Ao explorar o jogo, Flagamer explicou as configurações iniciais, como ajuste de sensibilidade do volante e configuração da tela para 16:9. Ele detalhou os diferentes níveis de dificuldade - Easy, Normal e Pro - e destacou que, mesmo no nível Easy, o jogo não 

In [30]:
output_texto.close()
with open("Y:/Transcription lives/data/output.txt", "a") as texto:
    texto.write('\n'+ response["choices"][0]["message"]["content"] + '\n')